# Final Project

# Exploratory data analysis of the Disney datasets

Author: Xuan Hong(Eyre)

# Introduction

## Question(s) of interests
In this analysis, I will be investigating a question associated with the collection of Disney datasets. I am interested in finding which animation director creates the most revenue gross for Disney. It is interesting because Disney is one of the most successful animation movie productions companies, and we can find some clues of how the animation industry changes over the years. We all know people have different tastes in the arts, especially the movies, but the economic benefits of a movie are straightforward.

My favourite Disney director is **[Byron Howard](https://en.wikipedia.org/wiki/Byron_Howard)**. He directed the [Zootopia](https://en.wikipedia.org/wiki/Zootopia), which won the 2016 Oscar and is also my favourite Disney animation. Although Howard is an amazing director, he is more an animator but a director for many years. This means that Howard hasn't directed a lot of movies yet. In order to make a more reasonable guess, I did some research and found a "[Top Animation Directors Ever](https://www.imdb.com/list/ls056336503/)" list. 


![](img/top_animation_directors.png)

The first is "[Walt Disney](https://en.wikipedia.org/wiki/Walt_Disney)", who is great but we can not count him because he is all the Disney movies producer. The second and third are "[Hayao Miyazaki](https://en.wikipedia.org/wiki/Hayao_Miyazaki)" and "[John Lasseter](https://en.wikipedia.org/wiki/John_Lasseter)", who are fabulous but not Disney director. The fourth is "[Ron Clements](https://en.wikipedia.org/wiki/Ron_Clements)" who directs "[The Little Mermaid](https://en.wikipedia.org/wiki/The_Little_Mermaid_(1989_film))", "[Hercules](https://en.wikipedia.org/wiki/Hercules_(1997_film))", "[The Princess and the Frog](https://en.wikipedia.org/wiki/The_Princess_and_the_Frog)" etc. and yes, a Disney director. Therefore, I would expect **Ron Clements** to be the animation director who creates the most revenue gross for Disney.


## Dataset description 

The below descripitions were taken directly from the [website](https://data.world/kgarrett/disney-character-success-00-16) where the datasets were obtained.

"Disney characters, box office success & annual gross income"
"What are the trends in the Walt Disney Studio’s box office data? How do certain characters contribute to the success or failure of a movie?"


The Disney dataset is composed of $5$ tables, `disney_movies_total_gross.csv`, `disney_revenue_1991-2016.csv`, `disney-characters.csv`, `disney-director.csv`, and `disney-voice-actors.csv`. Each table is stored in a `.csv` file and contains different information about Disney movies including its revenue, directors, release date, characters, and voice actors. I will be using the `disney_movies_total_gross` and `disney-director` tables formally described below:

* **disney_movies_total_gross.csv**
    * Disney movie box office gross and inflation adjustments.
* **disney-director.csv**
    * Only scraped the first director's name of each animation film.

# Methods and Results

Since Disney produces not only animation but also real actors' movies, I will need to clean the **disney_movies_total_gross** because I am only interested in animation directors. This means I will only consider the movies listed in **disney-director**.

However, before moving further, let us import the tables and do some basic visualizations.

In [1]:
# Lets import all the required libraries needed for this analysis
import altair as alt
import pandas as pd

# import all the required files
gross = pd.read_csv("data/disney_movies_total_gross.csv")
directors = pd.read_csv("data/disney-director.csv")

Let's see what the tables look like.

In [2]:
gross.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000","$920,608,730"


In [3]:
directors.head()

,name,director
0,Snow White and the Seven Dwarfs,David Hand
1,Pinocchio,Ben Sharpsteen
2,Fantasia,full credits
3,Dumbo,Ben Sharpsteen
4,Bambi,David Hand


Let's get some other information about the **gross** table.

In [4]:
gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   movie_title               579 non-null    object
 1   release_date              579 non-null    object
 2   genre                     562 non-null    object
 3   MPAA_rating               523 non-null    object
 4   total_gross               579 non-null    object
 5   inflation_adjusted_gross  579 non-null    object
dtypes: object(6)
memory usage: 27.3+ KB


The **gross** table has $579$ rows and $6$ columns. Every **movie_title** has a **release_date**, the number of its **total_gross** in the release year, and the number of **inflation_adjusted_gross**, which adjust the revenue regarding to the 2016 value. The **genre** and the **MPAA_rating** have some missing entries, but they do not relate to our analysis.

Let's get some other information about the **directors** table.

In [5]:
directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      56 non-null     object
 1   director  56 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


The **directors** table has $56$ rows with $2$ columns. Every movie has a **name**, and a **director**.

As a first visualization, lets look at the gross ranking of Disney movies from $1937$ to $2016$ regarding the $2016$ adjusted value. To do this, I will use the **gross** table. 

In [6]:
#covert the currency $ string in the "totall_gross" and "inflation_adjusted_gross" to float
gross[gross.columns[4:]] = gross[gross.columns[4:]].replace('[\$,]', '', regex=True).astype(float)

#sort the gross value based on the "inflation_adjusted_gross"
gross_rank = gross.sort_values(by="inflation_adjusted_gross",ascending=False)

#Reset the index so we can plot using altair
gross_rank = gross_rank.reset_index(drop=True)
gross_rank.head(10)

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,184925485.0,5.228953e+09
1,Pinocchio,"Feb 9, 1940",Adventure,G,84300000.0,2.188229e+09
2,Fantasia,"Nov 13, 1940",Musical,G,83320000.0,2.187091e+09
3,101 Dalmatians,"Jan 25, 1961",Comedy,G,153000000.0,1.362871e+09
4,Lady and the Tramp,"Jun 22, 1955",Drama,G,93600000.0,1.236036e+09
5,Song of the South,"Nov 12, 1946",Adventure,G,65000000.0,1.078511e+09
6,Star Wars Ep. VII: The Force Awakens,"Dec 18, 2015",Adventure,PG-13,936662225.0,9.366622e+08
7,Cinderella,"Feb 15, 1950",Drama,G,85000000.0,9.206087e+08
8,The Jungle Book,"Oct 18, 1967",Musical,Not Rated,141843000.0,7.896123e+08
9,The Lion King,"Jun 15, 1994",Adventure,G,422780140.0,7.616409e+08


Now that we have it in the proper format, we can generate a bar plot to visualize it.

In [7]:
# Use altair to generate a bar plot
gross_rank_plot = (
    alt.Chart(gross_rank, width=7000, height=300)
    .mark_bar(size=10)
    .encode(
        x=alt.X("movie_title:N", title="Disney Movie"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Gross Value (adjusted up to 2016)"),
    )
    .properties(title="Gross Ranking of Disney Movies from 1937 to 2016")
)
gross_rank_plot

alt.Chart(...)

From this plot, we know that the top $5$ animation is "Snow White and the Seven Dwarfs", "Pinocchio", "Fantasia", "101 Dalmatians", and "Lady and the Tramp". It looks like they are all from the last century almost $70$ years ago. Also, this plot is long and messy. In this case, I think I need to take a deeper look at the **gross** data more about its time.

Let's create a second visualization of the yearly gross value of Disney movies. Before doing that, I need to get the year value separately for every movie and rearrange the columns we need.

In [8]:
#get the year, month, day separately of each movie
dates = (gross['release_date'].str.split(' ', expand=True).rename(columns = {0:'month',
                                                                     1:'day',
                                                                     2:'year'}))

dates

,month,day,year
0,Dec,"21,",1937
1,Feb,"9,",1940
2,Nov,"13,",1940
3,Nov,"12,",1946
4,Feb,"15,",1950
...,...,...,...
574,Sep,"2,",2016
575,Sep,"23,",2016
576,Nov,"4,",2016
577,Nov,"23,",2016


In [9]:
#assign the seperate year to the gross DataFrame 
gross_year = gross.assign(year = dates['year'].astype(int))

#rearrange the gross_year columns
gross_year = gross_year.loc[:, ['year', 'movie_title', 'total_gross', 'inflation_adjusted_gross']]
gross_year

,year,movie_title,total_gross,inflation_adjusted_gross
0,1937,Snow White and the Seven Dwarfs,184925485.0,5.228953e+09
1,1940,Pinocchio,84300000.0,2.188229e+09
2,1940,Fantasia,83320000.0,2.187091e+09
3,1946,Song of the South,65000000.0,1.078511e+09
4,1950,Cinderella,85000000.0,9.206087e+08
...,...,...,...,...
574,2016,The Light Between Oceans,12545979.0,1.254598e+07
575,2016,Queen of Katwe,8874389.0,8.874389e+06
576,2016,Doctor Strange,232532923.0,2.325329e+08
577,2016,Moana,246082029.0,2.460820e+08


Now, let's group by 'year' and count the total number of the 'inflation_adjusted_gross' and sort it in a descending way. To do this, I will import and use the script I created with a custom function that takes in a data frame and groups it by a certain column and then sort it by the values we interest.

In [10]:
# import the custom script
import script as ps

# run it on the data
gross_year_group = ps.custom_sort(gross_year,'year','inflation_adjusted_gross')
gross_year_group

,year,inflation_adjusted_gross
0,1937,5.228953e+09
1,1940,4.375320e+09
2,2016,2.873393e+09
3,2015,2.495663e+09
4,1998,2.189031e+09
5,1995,2.188599e+09
6,2003,2.171245e+09
7,1996,2.157239e+09
8,1994,2.140691e+09
9,1999,1.981583e+09


Now that we have reasonable yearly values in the proper format, we can generate a bar plot to visualize it.

In [11]:
# Use altair to generate a bar plot
gross_year_plot = (
    alt.Chart(gross_year_group, width=700, height=300)
    .mark_bar()
    .encode(
        x=alt.X("year:Q", title="Year"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Gross Value(adjusted up to 2016)"),
    )
    .properties(title="Total Gross Revenue of Disney Movies from 1937 to 2016")
)
gross_year_plot

alt.Chart(...)

From the above plot, we can see the historical development of Disney. In 1937 and 1940, Disney made its highest value achievement. Then it slows down and creates one animation almost every $5$ years. Since `1980`, Disney started to create movies every year and get a high number gross every $3-4$ years. Generally, the number keeps growing from `1980` and doubles quickly in `1987`. It reaches its first peak around `1995` and decreases slightly from `2000` to `2012`. After that, the number increases again and reaches the third-highest gross in Disney history in `2016`.
This plot gives me more confidence in recent $30$ years animations.

As a third visualization, let's take a look at the number of animations directed by every director. To do this, I will use **directors** DataFrame.

Now let's group by 'director' name and count the frequency of the animation 'name'. 

In [12]:
# group by director and count the frequency of movies
animation_with_directors_group = pd.DataFrame(directors.groupby('director')['name'].count())

# Reset the index so we can plot using altair
animation_with_directors_group = animation_with_directors_group.reset_index()
animation_with_directors_group

,director,name
0,Art Stevens,1
1,Barry Cook,1
2,Ben Sharpsteen,2
3,Byron Howard,1
4,Chris Buck,2
5,Chris Sanders,1
6,Chris Williams,1
7,Clyde Geronimi,3
8,David Hand,2
9,Don Hall,1


Let's do a scatter plot of the directors with the most animations.

In [13]:
directors_plot = (
    alt.Chart(animation_with_directors_group, width=500, height=300)
    .mark_circle()
    .encode(
        x=alt.X("director:O", sort="y", title="Disney directors"),
        y=alt.Y("name:Q", title="Numbers of animations"),
    )
    .properties(title="Numbers of animations directed by Disney directors")
)
directors_plot

alt.Chart(...)

It seems that both our expectation **Ron Clements**, and a director **[Wolfgang Reitherman](https://en.wikipedia.org/wiki/Wolfgang_Reitherman)**, who directed "[101 Dalmatians](https://en.wikipedia.org/wiki/One_Hundred_and_One_Dalmatians)", "[The Jungle Books](https://en.wikipedia.org/wiki/The_Jungle_Book_(1967_film))" etc. have created $7$ animations. What a fierce competition! 

Now, it is time to answer our original questions. First, I need to change the column "name" in **directors** to "movie_title" to match the **gross** for merging the data.

In [14]:
# change the column name to match
directors = directors.rename({'name': 'movie_title'}, axis=1) 
directors

,movie_title,director
0,Snow White and the Seven Dwarfs,David Hand
1,Pinocchio,Ben Sharpsteen
2,Fantasia,full credits
3,Dumbo,Ben Sharpsteen
4,Bambi,David Hand
5,Saludos Amigos,Jack Kinney
6,The Three Caballeros,Norman Ferguson
7,Make Mine Music,Jack Kinney
8,Fun and Fancy Free,Jack Kinney
9,Melody Time,Clyde Geronimi


Ok, let's merge it with the **gross** dataframe and we will only focus on the movies listed in **directors** since we want the animation directors.

In [15]:
directors_gross_merged = pd.merge(gross, directors, on="movie_title")
directors_gross_merged

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,director
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,184925485.0,5.228953e+09,David Hand
1,Pinocchio,"Feb 9, 1940",Adventure,G,84300000.0,2.188229e+09,Ben Sharpsteen
2,Fantasia,"Nov 13, 1940",Musical,G,83320000.0,2.187091e+09,full credits
3,Cinderella,"Feb 15, 1950",Drama,G,85000000.0,9.206087e+08,Wilfred Jackson
4,Cinderella,"Mar 13, 2015",Drama,PG,201151353.0,2.011514e+08,Wilfred Jackson
5,Lady and the Tramp,"Jun 22, 1955",Drama,G,93600000.0,1.236036e+09,Hamilton Luske
6,Sleeping Beauty,"Jan 29, 1959",Drama,NaN,9464608.0,2.150583e+07,Clyde Geronimi
7,101 Dalmatians,"Jan 25, 1961",Comedy,G,153000000.0,1.362871e+09,Wolfgang Reitherman
8,101 Dalmatians,"Nov 27, 1996",Comedy,G,136189294.0,2.587289e+08,Wolfgang Reitherman
9,The Sword in the Stone,"Dec 25, 1963",Adventure,NaN,22182353.0,1.538708e+08,Wolfgang Reitherman


We can see some duplicated rows in "movie_title" like "Cinderella", "The Jungle Book", and "101 Dalmatians". When we check the release date, they are all different. The first one of these movies is animation, the rest are all real-actor movies filmed in recent years. We will need to delete them in case it affects our results.

In [19]:
# Drop rows with duplicted movie_title 
directors_gross_merged = directors_gross_merged.drop_duplicates(subset="movie_title")

# Reset the index to get the right order
directors_gross_merged = directors_gross_merged.reset_index(drop=True)

Finally, let's group the directors and sum up the revenue of all the movies directed by every director to find who creates the most revenue. Here, we can use our custom sort function to save time again!

In [17]:
# run custom sort on the data
directors_gross_merged_group = ps.custom_sort(directors_gross_merged,'director','inflation_adjusted_gross')
directors_gross_merged_group

,director,inflation_adjusted_gross
0,David Hand,5.228953e+09
1,Wolfgang Reitherman,2.721260e+09
2,Ben Sharpsteen,2.188229e+09
3,full credits,2.187091e+09
4,Ron Clements,1.318950e+09
5,Hamilton Luske,1.236036e+09
6,Wilfred Jackson,9.206087e+08
7,Roger Allers,7.616409e+08
8,Chris Buck,6.988974e+08
9,Gary Trousdale,6.791946e+08


It seems that in fact it is **[David Hand](https://en.wikipedia.org/wiki/David_Hand_(animator))**  is the director who creates the most revenue. Our expectation **Ron Clements** is the forth if we ignore the "all credits" one.

In [18]:
# Visualize the gross created by the directors using a bar plot.
directors_gross_plot = (
    alt.Chart(directors_gross_merged_group, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("director:N", title="Disney Directors", sort="-y"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Gross Revenue of their Animations(adjusted up to 2016)"),
    )
    .properties(title="Total Gross Revenue of Disney Animations Created by Every Director")
)
directors_gross_plot

alt.Chart(...)

# Discussions

In this project, I analyzed the Disney dataset and tried to find which director creates the most revenue. Before answering this question, I did some exploratory data analysis to see which movies gain the most revenue, how is the total revenue of Disney movies changes over the year. Generally, there is an increasing trend in the total number since $1980$. However, the animations released in $1930-1960$ are so popular and gorgeous and win the top revenue. That nearly determined the result because the top three directors are all active in $1930-1960$. The top first, **David Hand** is the director of "[Snow White and the Seven Dwarfs](https://en.wikipedia.org/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film))" and "[Bambi](https://en.wikipedia.org/wiki/Bambi)". The second, **Wolfgang Reitherman**, I have introduced that he directed "[101 Dalmatians](https://en.wikipedia.org/wiki/One_Hundred_and_One_Dalmatians)", "[The Jungle Books](https://en.wikipedia.org/wiki/The_Jungle_Book_(1967_film))" etc. The third, **[Ben Sharpsten](https://en.wikipedia.org/wiki/Ben_Sharpsteen)**, is the director of "[Pinocchio](https://en.wikipedia.org/wiki/Pinocchio_(1940_film))" and "[Dumbo](https://en.wikipedia.org/wiki/Dumbo)".


My assumption **Ron Clements** is still the director who creates the most revenue in the recent $40$ years since $1980$. If we ignore the effects of the inflation-adjusted rates, he can indeed win the first price. However, it will be unfair to compare the revenue without the adjustment. Let's still follow the inflation adjust gross in this project. 

In this project, I did a lot of research on animations, and they bring my memory back to when I was a kid. Regardless of the inflation rate, one of the reasons that the animations in $1930-1960$  got that much achievement is because people at that time were longing for pure and childish happiness. Those directors, in a way, create some lovely dreams for both children and adults.

Another question that could be looked at given this dataset is to find the most welcome genre movies. One could look at which genre has created the most revenue over the years. This is interesting because Disney has produced different types of movies. It is meaningful for us to have some ideas about which genre is the most popular one. We can also see how the audience's taste changes over the years. Will the audience in $70$ years ago like a different kind of movies with us. Or are people the same? This might also influent Disney's filming decision.

# References

Not all the work in this notebook is original. Some parts were borrowed from online resources. I take no credit for parts that are not mine. They were solely used for illustration purposes. Let's give to **Ceasar** what belongs to **Ceasar**.

## Resources used



* [Data Source](https://data.world/kgarrett/disney-character-success-00-16)
    * This Disney database used in this work was curated by **Kelly Garrett**.
* [Data Visualization & Question Of Interest](https://public.tableau.com/app/profile/avinashreddymunnangi/viz/HistoryofDisneyMovies1937-2016/HistoryofDisneyMovies1937-2016)
    * Inspiration for generating the plotting the total revenue over the years and consider the inflation rate influence was taken from **Avinash Reddy Munnangi**.
* [WikiPedia](https://en.wikipedia.org/wiki/Main_Page)
     * Introductions of the animations and directors I mentioned.
* [IMDB](https://www.imdb.com/list/ls056336503/)
     * Inspiration of the achievement of animators in the 30s-60s.